# Part I. One-sided finite differences

Write a function, `deriv`, which computes a derivative of its argument at a given point, $x$, using a one-sided finite difference rule with a given step side $h$, with the approximation order of $O(h^2)$. 

In [1]:
def deriv(f, x, h):
    """ Compute a derivative of `f` at point `x` with step size `h`.
    
    Compute the derivative using the one-sided rule of the approximation order of $O(h^2)$.
    
    Parameters
    ----------
    f : callable
        The function to differentiate
    x : float
        The point to compute the derivative at.
    h : float
        The step size for the finite different rule.
        
    Returns
    -------
    fder : derivative of f(x) at point x using the step size h.
    """
    return ((f(x + h/2) - f(x - h/2)) / h)

#### Test I.1

Test your function on a simple test case: differentiate $f(x) = x^3$ at $x=0$. Comment on whether your results are consistent with the expected value of $f'(x) = 0$ and on an expected scaling with $h\to 0$.

 (10% of the total grade)

In [6]:
x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(lambda x: x**3, x, h)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- 2.5e-05
0.001000 -- 2.5e-07
0.000100 -- 2.5e-09
0.000010 -- 2.5e-11


При изменении h в 10 раз, ошибка изменяется в 100 раз, как и ожидалось.

### Test I.2

Now use a slightly more complicated function, $f(x) = x^2 \log{x}$, evaluate the derivative at $x=1$ using your one-sided rule and a two-point one-sided rule. Roughly estimate the value of $h$ where the error stops decreasing, for these two schemes. 
(15% of the total grade)

In [3]:
from math import log

def f(x):
    return x**2 * log(x)
    
def fder(x):
    return x * (2.*log(x) + 1)

In [34]:
def three_point_deriv(f, x, h):
    f0 = f(x)
    fh = f(x + h)
    f2h = f(x + 2 * h)
    return ((4 * fh - 3 * f0 - f2h) / (2 * h))

x = 1
for h in [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]:
    err = deriv(f, x, h) - fder(x)
    print("%5f -- %7.4g" % (h, err))

print('\n')

x = 1
for h in [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]:
    err = three_point_deriv(f, x, h) - fder(x)
    print("%5f -- %7.4g" % (h, err))

0.010000 --  -1.071
0.001000 --  -1.071
0.000100 --  -1.071
0.000010 --  -1.071
0.000001 --  -1.071
0.000000 --  -1.071


0.010000 --  -1.071
0.001000 --  -1.071
0.000100 --  -1.071
0.000010 --  -1.071
0.000001 --  -1.071
0.000000 --  -1.071


### Test I.3 

Now try differentiating $x^2 \log(x)$ at $x=0$. Use the three-point one-sided rule. Note that to evaluate the function at zero, you need to special-case this value. Check the scaling of the error with $h$, explain your results. 
(25% of the total grade)

In [36]:
def f(x):
    if x == 0:
        # the limit of $x^2 log(x)$ at $x-> 0$ is zero, even though log(x) is undefined at x=0
        return 0.0
    else:
        return x**2 * log(x)
    
def fder(x):
    if x == 0:
        return 0.0
    else:
        return x*(2*log(x) + 1)

x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = three_point_deriv(f, x, h) - fder(x)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- -0.01386
0.001000 -- -0.001386
0.000100 -- -0.0001386
0.000010 -- -1.386e-05


... ENTER YOUR EXPLANATION HERE ...

# Part II. Midpoint rule 

Write a function which computes a definite integral using the midpoint rule up to a given error, $\epsilon$. Estimate the error by comparing the estimates of the integral at $N$ and $2N$ elementary intervals. 

In [23]:
def midpoint_rule(func, a, b, eps):
    """ Calculate the integral of f from a to b using the midpoint rule.
    
    Parameters
    ----------
    func : callable
        The function to integrate.
    a : float
        The lower limit of integration.
    b : float
        The upper limit of integration.
    eps : float
        The target accuracy of the estimate.
        
    Returns
    -------
    integral : float
        The estimate of $\int_a^b f(x) dx$.
    """
    
    prev_res = 0
    n = 1
    d = (b - a) / n
    for i in range (n):
        prev_res += func(a + (i + 0.5) * d) * d
    while (True):
        n = 2 * n
        d = (b - a) / n
        res = 0
        for i in range (n):
            res += func(a + (i + 0.5) * d) * d
        
        if ((res - prev_res)**2 < eps**2):
            return (res, n)
        prev_res = res
        
        

### Test II.1

Test your midpoint rule on a simple integral, which you can calculate by paper and pencil.

Compare the rate of convergence to the expected $O(N^{-2})$ scaling by studying the number of intervals required for a given accuracy $\epsilon$.

Compare the numerical results to the value you calculated by hand. Does the deviation agree with your estimate of the numerical error?
(20% of the total grade)


In [24]:
f = lambda x: x**2
fint, niter = midpoint_rule(f, 0, 1, 1e-3)
print (fint)
print (niter)

0.3330078125
16


### Test II.2

Now use your midpoint rule to compute the value of

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx
$$

up to a predefined accuracy of $\epsilon=10^{-4}$.

Note that the integral contains an integrable singularity at the lower limit. Do calculations two ways: first, do a straightforward computation; next, subtract the singularity. Compare the number of iterations required to achieve the accuracy of $\epsilon$.

(30% of the total grade)

In [33]:
import numpy as np

f = lambda x: (np.sin(x**0.5))/x
fint, niter = midpoint_rule(f, 0, 1, 1e-4)
print (fint)
print (niter)

f = lambda x: ((np.sin(x**0.5))/x - x**(-0.5))
fint, niter = midpoint_rule(f, 0, 1, 1e-4)
print (fint + 2) # 2 это интеграл вычтенной части
print (niter)

1.891957289204461
8388608
1.892113005639548
32
